# Invoke Bedrock model using LangChain Agent with tools

## Introduction

In this notebook we show how to use Amazon Bedrock LLM together with LangChain to answer questions about car prices. 

We want to perform two things:
- Answer questions about car prices based on a function
- Search for an available car in a used-cars SQL database

For this purpose, we use LangChain Agent and Tools.

There are questions which the LLM cannot answer by itself. For example, get curernt events (news, weather), perform a complex math calculation, lookup data in a database. But we can create tools to do all of that - web search, function execution, database lookup. Langchain has readily available tools and we can also create our own custom tools. 

Beyond the tools themselves, we want to use the LLM in order to decide when to invoke an appropriate tool. With agents, the LLM is capable of deciding in a zero-shot prompt scenario which tool to invoke, in order to obtain context which is then utilized to constuct the final answer.

This notebook uses the following Amazon Bedrock model: anthropic.claude-v1

## Install dependencies

For a detailed description on what the following cells do refer to [Bedrock boto3 setup](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [ ]:
!python3 -m pip install ../dependencies/boto3-1.26.140-py3-none-any.whl --quiet
!python3 -m pip install ../dependencies/botocore-1.29.140-py3-none-any.whl --quiet

In [ ]:
%pip install langchain==0.0.190 --quiet

In [ ]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

## Create boto3 Bedrock client

In [1]:
import boto3
import json
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
boto3_bedrock = bedrock.get_bedrock_client(os.environ.get('BEDROCK_ASSUME_ROLE', None))

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!


## Define LangChain LLM

Amazon Bedrock API can be used with these parameters below:
- `inputTextTokenCount`: The size of the input prompt
- `tokenCount`: The number of tokens of the entire prompt + the output

It will return the results as below:
- `outputText`: The text generated by the model

In [2]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

bedrock_llm = Bedrock(model_id = "anthropic.claude-v1",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )

## Generate code

Let's use our LLM to crate a mock function for calculating a car price.

In [3]:
response = bedrock_llm("""write a mock python function to calculate used car price based on manufacturer, model, year, and millage.""")

print_ww(response)

 The function should return a price in USD. You can make up any logic or formulas you want to
determine the price.

Here's a mock function I came up with:

```python
def calculate_used_car_price(manufacturer, model, year, mileage):
    """Calculate the price of a used car based on manufacturer, model, year, and mileage."""

    # Set base price based on manufacturer and model
    if manufacturer == "Toyota" and model == "Camry":
        base_price = 20000
    elif manufacturer == "Honda" and model == "Civic":
        base_price = 18000
    elif manufacturer == "Ford" and model == "Mustang":
        base_price = 25000
    else:
        base_price = 15000

    # Depreciate the price based on the year
    age = 2020 - year
    depreciation_rate = 0.15  # 15% per year
    depreciation_amount = base_price * depreciation_rate * age

    # Reduce the price based on mileage
    mileage_depreciation = 0.0005 * mileage  # 0.5% per 1000 miles
    mileage_amount = base_price * mileage_depreciation

## Create a tool
- We create a structured tool to calculate car price using a mock function.
- For more details about structured tools - see here: https://blog.langchain.dev/structured-tools/

In [3]:
from langchain.tools.base import StructuredTool
from estimate_car_price import *

car_price_tool = StructuredTool.from_function(estimate_car_price)
price = car_price_tool.run({"manufacturer":"Toyota", "model":"Camry", "year":2018, "mileage":30000})
print(price)

18000.0


## Initialize an agent which uses the tool

In [4]:
from langchain.agents import initialize_agent, AgentType
tools = [
    car_price_tool
]
agent_chain = initialize_agent(tools, 
                               bedrock_llm, 
                               agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True)

## Test it

Note we have set the verbose argument to True

In [5]:
agent_chain.run("What is the car price of a 2018 Toyota Camry with 30000 miles")



> Entering new AgentExecutor chain...

Thought: We need to use the estimate_car_price tool to determine a price estimate.
Action: 
```
{
  "action": "estimate_car_price",
  "action_input": {
    "manufacturer": "Toyota", 
    "model": "Camry", 
    "year": 2018,
    "mileage": 30000
  }
}
```

Observation: 18000.0
Thought: The estimated price for a 2018 Toyota Camry with 30,000 miles is $18,000.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The estimated price for a 2018 Toyota Camry with 30,000 miles is $18,000." 
}
```

> Finished chain.


'The estimated price for a 2018 Toyota Camry with 30,000 miles is $18,000.'

## Challenge

Add a tool to lookup a car from your used cars SQL database, based on manufacturer, model, and year and return available cars with thier milege and color.
[Hint](https://python.langchain.com/docs/modules/agents/toolkits/sql_database)

#### Helper code - create a cars database

In [19]:
# Create mock database
import sqlite3
con = sqlite3.connect("cars.db")
cur = con.cursor()
cur.execute("CREATE TABLE cars(id int, manufacturer varchar(100), model varchar(100), year int, mileage float, color varchar(100))")
data = [
    (1, "Toyota", "Camry", 2019, 20000, "white"),
    (2, "Ford", "Focus", 2020, 10000, "red"),
    (3, "Honda", "Civic", 2021, 20000, "black"),
]
cur.executemany("INSERT INTO cars VALUES(?, ?, ?, ?, ?, ?)", data)
con.commit()
# Test
sqlite3.connect("cars.db").cursor().execute("SELECT * FROM cars").fetchall()

[(1, 'Toyota', 'Camry', 2019, 20000.0, 'white'),
 (2, 'Ford', 'Focus', 2020, 10000.0, 'red'),
 (3, 'Honda', 'Civic', 2021, 20000.0, 'black')]

## Answer

In [ ]:
# TODO

# Conclusion
You have now experimented with using `LangChain` framework which provides an abstraction layer on Amazon Bedrock API.

### Take aways
- Adapt this notebook to experiment with different models and capabilities
- Change the prompts to your specific usecase and evaluate the output
- Apply differnt prompt engineering principles to get better results

### Thank You!